In [2]:
import numpy as np
%matplotlib notebook
from matplotlib import pyplot as plt
from networkx import grid_graph
%pylab inline
%load_ext autoreload
%autoreload 2

from cvf20.unionfind import UnionFind as UF

Populating the interactive namespace from numpy and matplotlib


# 1. Generalized All-Pairs Shortest Path Problem

In [105]:
def generalized_shortest_path(graph, gen_sum, gen_mult):
    """
    Parameters
    ---------

    graph    :     ndarray
        Quadratic matrix representing the edge weights between the nodes.
    gen_sum    :    function
        Function representing generalized sum, that takes list as arg.
    gen_mult    :    function
        Function representing generalized multiplication, that takes 2 elements of monoid as arg.
    """
    assert graph.shape[0] == graph.shape[1]
    counter = 0
    n = graph.shape[0]
    previous = np.ones_like(graph) * np.inf
    result = np.copy(graph)
    while not np.equal(previous, result).all():
        counter += 1
        previous = np.copy(result)
        for i in range(n):
            for j in range(n):
                result[i][j] = gen_sum([gen_mult(graph[i][x], previous[x][j]) for x in range(n)])
        # modification for c)
        if any([np.diagonal(result) < 0]):
            print('negative cycle is present')
            return
    print(f'needed {counter} multiplications')
    return result

In [106]:
graph = np.array([[0, 6, np.inf, np.inf, np.inf],
                  [np.inf, 0, np.inf, np.inf, 2],
                  [np.inf, 3, 0, np.inf, np.inf],
                  [1, 5, 8, 0, np.inf],
                  [np.inf, np.inf, np.inf, 4, 0]])

### a) All-pairs shortest path

In [107]:
def minimum(l):
    return np.min(l)


def add(a, b):
    return a + b

shortest_path = generalized_shortest_path(graph, minimum, add)
print(f'shortest_path:\n{shortest_path}')

needed 4 multiplications
shortest_path:
[[ 0.  6. 20. 12.  8.]
 [ 7.  0. 14.  6.  2.]
 [10.  3.  0.  9.  5.]
 [ 1.  5.  8.  0.  7.]
 [ 5.  9. 12.  4.  0.]]


### b) All-pairs minimax path

In [108]:
def minimum(l):
    return np.min(l)


def maximum(a, b):
    return np.max([a,b])

shortest_path = generalized_shortest_path(graph, minimum, maximum)
print(f'shortest_path:\n{shortest_path}')

needed 4 multiplications
shortest_path:
[[0. 6. 8. 6. 6.]
 [4. 0. 8. 4. 2.]
 [4. 3. 0. 4. 3.]
 [1. 5. 8. 0. 5.]
 [4. 5. 8. 4. 0.]]


### c) Negative cycles

If any of the diagonal elements is < 0, a negative cycle is present in the graph.

# 2. Distance Transform Watershed using Fiji

### b.) Distance Transform Watershed
Use the code block below to load the image you generated with Fiji and plot it overlayed with the original raw image.

In [ ]:
from cvf20.utils import plot_segm
raw = plt.imread("data/raw.png")
WS = plt.imread("data/distance-transform-WS.tif")

f, ax = plt.subplots(ncols=1, nrows=1, figsize=(15,15))
plot_segm(ax, WS, with_background_label=True, alpha_labels=0.4, background_image=raw)
plt.show()

### c.) Segmenting boundaries 
Use the code block below to load the image you generated with Fiji and plot it overlayed with the original raw image.

In [ ]:
from cvf20.utils import plot_segm
raw = plt.imread("data/raw.png")
WS = plt.imread("data/distance-transform-WS-no-boundaries.tif")

f, ax = plt.subplots(ncols=1, nrows=1, figsize=(15,15))
plot_segm(ax, WS, with_background_label=True, alpha_labels=0.4, background_image=raw)
plt.show()

# 3. Bonus: Connected Component Labeling

### b) Connected components on graphs
Use the code block below to test your implementation on a randomly generated graph with 100 vertices. For every pair of nodes, we introduce an edge with probability 0.02. This is usually known as Erdős-Rényi graph or binomial graph. 

You should find that the graph has one very big connected component and several small ones.

In [10]:
from networkx.algorithms.components import number_connected_components
from networkx.generators.random_graphs import fast_gnp_random_graph

# We now generate a graph with 100 vertices. For every pair of nodes, we introduce an edge with probability 0.02
# This is usually known as Erdős-Rényi graph or binomial graph.
G = fast_gnp_random_graph(100, 0.02)
nodes = [n for n in G.nodes()]
edges = [e for e in G.edges()]

########################### 
# Your code starts here:
###########################

uf = UF()
for n in nodes:
    uf.add(n)

for u, v in edges:
    uf.union(u, v)

    
parents = np.zeros((len(nodes)), dtype='int32')
for n in nodes:
    parents[n] = uf.find(n)
    

size_connected_components = np.unique(parents, return_counts=True)[1]
#size_connected_components = uf.n_comps 

########################### 
# Your code ends here
###########################


if size_connected_components is not None:
    assert size_connected_components.shape[0] == number_connected_components(G)
    print("Size connected components: ", size_connected_components)
    print("Biggest connected component: {} vertices (out of 100)".format(size_connected_components.max()))

Size connected components:  [78  1  1  1  1  1  3  1  2  1  2  1  1  1  1  1  1  1  1]
Biggest connected component: 78 vertices (out of 100)
